In [46]:
import praw
import pandas as pd
# from praw.models import MoreComments #This import is needed is used only if we want to limit the comments we scrape

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps

#This is the tutorial I used to set up the web scraping using PRAW
#https://www.geeksforgeeks.org/python/scraping-reddit-using-python/

# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

# df1 = pd.DataFrame({
#     'post_id': ['p1', 'p2', 'p3'],
#     'title': ['Title A', 'Title B', 'Title C'],
#     'author': ['user1', 'user2', 'user3']
# })

# df2 = pd.DataFrame({
#     'comment_id': ['c1', 'c2', 'c3', 'c4', 'c5'],
#     'post_id': ['p1', 'p1', 'p2', 'p2', 'p4'],  # Note: p4 doesn't exist in df1
#     'comment': ['Great post!', 'Thanks!', 'Interesting', 'I agree', 'Nice!']
# })
# outer_merged = df1.merge(df2, on='post_id', how='outer')
# print(outer_merged)

In [47]:
consulting_posts = subreddit.search('clubs', limit=None)

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [48]:
for post in consulting_posts:
    if len(post.selftext) > 20:
        #The title of the post
        posts_dict['Title'].append(post.title)
        #The text inside of the post
        posts_dict['Post Text'].append(post.selftext)
        #Unique ID of each post
        posts_dict['ID'].append(post.id)
        #The scoure of a post
        posts_dict['Score'].append(post.score)
        #Total number of comments inside the post
        posts_dict['Total Comments'].append(post.num_comments)
        #URL of each post
        posts_dict['Post URL'].append(post.url)
    else:
        continue

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Looking for people to start a bridge troll clu...,"Hello, \n\nI was wondering if anyone else woul...",1nfcxn2,262,36,https://www.reddit.com/r/berkeley/comments/1nf...
1,Heartbreaking club experience as a freshman,I applied to 9 tech clubs (i’m an eecs major) ...,1nh1egm,129,52,https://www.reddit.com/r/berkeley/comments/1nh...
2,Berkeley club decisions today got me like:,[Berkeley club decisions today got me like:](h...,1na5tea,69,50,https://www.reddit.com/r/berkeley/comments/1na...
3,Potential nepotism in prestigious clubs is sic...,Title. So exhausted of this kinda stuff. Heard...,1nbpwed,105,36,https://www.reddit.com/r/berkeley/comments/1nb...
4,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,269,112,https://www.reddit.com/r/berkeley/comments/16o...
...,...,...,...,...,...,...
205,How to get into Consulting clubs?,Hello! I'm going to be a sophomore next year a...,1ek2g0a,5,6,https://www.reddit.com/r/berkeley/comments/1ek...
206,2nd Year Looking for Clubs,"Hello everyone, \n\nI'm a sophomore looking fo...",1g00k5q,3,2,https://www.reddit.com/r/berkeley/comments/1g0...
207,"What is a class, club or experience on campus ...","Also, what are some good non-engineering, non-...",14q0rjm,38,21,https://www.reddit.com/r/berkeley/comments/14q...
208,Social club for grieving students at Cal,Hi! I am a part of a group of grieving Cal stu...,1ft54r5,28,0,https://forms.gle/Qx3U8xKf7v31WXsHA


In [49]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#Generate the CSV file that contains all the posts about Berkeley clubs
# comments_df.to_csv('my_dataframe.csv', index=False) # index=False prevents writing the DataFrame index as a column

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Don’t forget riddles,63
1,"if you’re deadass, i will absolutely join",50
2,"I'm actually serious about starting it, if som...",48
3,"definitely don’t go barefoot under a bridge, t...",16
4,Do we get to eat billy goats?,15
...,...,...
1038,I wanna do like a Krav Maga street style self ...,11
1039,"y'all lmk if we doin this, i may be a waitlist...",2
1040,"Down… but I’m a complete beginner, how would i...",2
1041,Would love to join! complete beginner here but...,2


In [50]:
new_df = pd.concat([consulting_club_posts['Post Text'], comments_df['Comment']], axis=1)
new_df

,Post Text,Comment
0,"Hello, \n\nI was wondering if anyone else woul...",Don’t forget riddles
1,I applied to 9 tech clubs (i’m an eecs major) ...,"if you’re deadass, i will absolutely join"
2,[Berkeley club decisions today got me like:](h...,"I'm actually serious about starting it, if som..."
3,Title. So exhausted of this kinda stuff. Heard...,"definitely don’t go barefoot under a bridge, t..."
4,Over the past month on this subreddit there ha...,Do we get to eat billy goats?
...,...,...
1038,NaN,I wanna do like a Krav Maga street style self ...
1039,NaN,"y'all lmk if we doin this, i may be a waitlist..."
1040,NaN,"Down… but I’m a complete beginner, how would i..."
1041,NaN,Would love to join! complete beginner here but...


In [51]:
# #Loading the stored dataframes from the previous notebook using the pickle module
# consulting_club_posts.to_pickle('consulting_club_posts.pkl')
# comments_df.to_pickle('comments_df.pkl')

In [52]:
new_df.to_csv('posts_and_comments.csv', index=False)

In [53]:
# This is test code if nothing is working
# #Display the name of the subreddit
# print("Display Name:", subreddit.display_name)

# #Display the title of the subreddit
# print("Title:", subreddit.title)

# #Display the description of the subreddit
# print("Description:", subreddit.description)

In [54]:
# subreddit._fetch()

# print(sorted(vars(subreddit).keys()))

# print("Display Name:", subreddit.display_name)

# try:
#     print("Title:", subreddit.unsubscribe)
# except Exception as e:
#     print("Error:", e)

# print("Title:", subreddit.title)

# print("Description:", subreddit.description)